# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.128252e+02     1.542582e+00
 * time: 0.6574521064758301
     1     1.130992e+01     9.212911e-01
 * time: 1.917598009109497
     2    -1.182032e+01     1.159881e+00
 * time: 2.542917013168335
     3    -3.402971e+01     8.636257e-01
 * time: 3.4161479473114014
     4    -4.749725e+01     6.946539e-01
 * time: 4.434570074081421
     5    -5.668621e+01     2.912351e-01
 * time: 5.439497947692871
     6    -5.944965e+01     2.923670e-01
 * time: 6.024939060211182
     7    -6.065879e+01     1.114434e-01
 * time: 6.615612030029297
     8    -6.126003e+01     4.957241e-02
 * time: 7.210827112197876
     9    -6.156892e+01     3.605370e-02
 * time: 7.806560039520264
    10    -6.182560e+01     2.862404e-02
 * time: 8.418776035308838
    11    -6.198914e+01     2.503343e-02
 * time: 9.027704954147339
    12    -6.209479e+01     1.621772e-02
 * time: 9.655018091201782
    13    -6.214216e+01     1.337455e-02
 * time: 10.259231090545654
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671068
    AtomicLocal         -18.8557620
    AtomicNonlocal      14.8522596
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485360 
    Xc                  -19.3336816

    total               -62.261666459705
